# Tutorial: Getting optimized conformers and energies

The most basic usage of Auto3D is to get the optimized low-energy conformers. The input is just a `smi` or `SDF` file, the output is an `SDF` file containing the conformers and its energies. `Auto3D` provides flexibility in choosing isomer engine, optimization engine and other properties, but the whole process could be done with as little as 5 lines of code.

In [1]:
import os, sys
root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))

import Auto3D
#options is a function that takes in all user-specified parameters, then return arguments for the main function
#main function takes in the arguments from the options function, then actually runs the Auto3D modules
from Auto3D.auto3D import options, main

In [2]:
#Always ensure that you have the latest version
print(Auto3D.__version__)

2.2.6


# Generate low-energy 3D structures with Auto3D

In [4]:
if __name__ == "__main__":
    path = os.path.join(root, "example/files/smiles.smi")  # You can specify the path to your file here
    args = options(path, k=1, use_gpu=False)   #specify the parameters for Auto3D 
    out = main(args)            #main acceps the parameters and run Auto3D
    print(out)

Checking input file...
	There are 4 SMILES in the input file /Users/liu5/Documents/Auto3D_pkg/example/files/smiles.smi. 
	All SMILES and IDs are valid.
Suggestions for choosing isomer_engine and optimizing_engine: 
	Isomer engine options: RDKit and Omega.
	Optimizing engine options: ANI2x, ANI2xt and AIMNET.
The available memory is 16 GB.
The task will be divided into 1 jobs.
Job1, number of inputs: 4


Isomer generation for job1
Enumerating cis/tran isomers for unspecified double bonds...
Enumerating R/S isomers for unspecified atomic centers...
Removing enantiomers...
Enumerating conformers/rotamers, removing duplicates...


100%|██████████| 4/4 [00:00<00:00, 19.43it/s]




Optimizing on job1
Preparing for parallel optimizing... (Max optimization steps: 5000)
Total 3D conformers: 39


 10%|▉         | 499/5000 [03:35<21:02,  3.57it/s]

Total 3D structures: 39  Converged: 23   Dropped(Oscillating): 0    Active: 16


 20%|██        | 1000/5000 [05:01<07:39,  8.70it/s]

Total 3D structures: 39  Converged: 35   Dropped(Oscillating): 0    Active: 4


 28%|██▊       | 1407/5000 [05:38<14:25,  4.15it/s]


Optimization finished at step 1408:   Total 3D structures: 39  Converged: 39   Dropped(Oscillating): 0    Active: 0
Begin to select structures that satisfy the requirements...
Energy unit: Hartree if implicit.
Program running time: 6 minute(s)
Output path: /Users/liu5/Documents/Auto3D_pkg/example/files/20231220-112847-937903_smiles/smiles_out.sdf
/Users/liu5/Documents/Auto3D_pkg/example/files/20231220-112847-937903_smiles/smiles_out.sdf


In the above example, Auto3D searches and optimizes 39 conformers, then returns the low-energy conformers using 6 minutes on a Intel MacBook. This time can be reduced to ~2 minutes if the job is run on a Nvidia RTX3090 GPU. The total running time also depends on the choice of different `optimizing_engine`:


| Optimizing_engine    | Elements Covered                                           | Charge                                | Training Dataset Size | Target DFT       | Empirical  accuracy and speed                                |
|----------|------------------------------------------------------------|---------------------------------------|-----------------------|------------------|----------------------------------------|
| ANI2x   | H, C, N, O, F, S, Cl                                       | only neutral molecules                | ~ 4.5M                | ωB97X/6-31G*     | accurate, very fast                 |
| ANI2xt  | H, C, N, O, F, S, Cl                                       | only neutral molecules, with enhancement for tautomers                | ~ 13M                 | B97-3c           | accurate, ultra fast               |
| AIMNET (default)   | H, B, C, N, O, F, Si, P, S, Cl, As, Se, Br, I              | both neutral and charged molecules   | ~ 20 M                  | wB97M/Def2-TZVPP           | very accurate, fast |

For the above example, `ANI2xt` is about 8 times faster than `AIMNet`, using 16 seconds on a RTX3090 GPU. `ANI2x` is about 2 times faster than `AIMNET`, using about 1 minute on a RTX3090 GPU. The speed difference is more abvious on larger jobs. As for accuracy, all models are accurate. The difference is only obvious when we do edge examples.

Note the `k=1` argument specifies that only the lowest energy conformer for each molecule will be kepted int the final output. If `k=3`, the each molecule will have at most 3 conformers, and their relative energies will be stored in the SDF file, too. Here is a diagram demonstrate the argument `k`.


<figure>
    <img src="./files/k.png" alt="illustraction of the k parameter" style="width: 800px;" />
</figure>

Note that you can also use `window=x` instead of setting `k=1`. The argument `window` tells Auto3D to keep all conforers whose energies are at most `x kcal/mol` higher than the lowest-energy of that molecule. `window` and `k` are mutual exclusive, so you can only set either `window` or `k`. 

# Running the job in CLI

You can also run the above job with CLI:

```
cd <replace with your path_folder_with_auto3D.py>
python auto3D.py "example/files/smiles.smi" --k=1
```

You can find `auto3D.py` at [here](https://github.com/isayevlab/Auto3D_pkg/blob/main/auto3D.py)

# Running the job in CLI with a yaml file
The parameter can be provided via a yaml file. So the above example is equivalent to
```
cd <replace with your path_folder_with_auto3D.py>
python auto3D.py parameters.yaml
```
You can find an example `parameters.yaml` at [here](https://github.com/isayevlab/Auto3D_pkg/blob/main/parameters.yaml)